In [1]:
import os
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import cv2

input_data_path = '/kaggle/input/face-mask-detection/images'
annotations_path = "/kaggle/input/face-mask-detection/annotations"
images = [*os.listdir("/kaggle/input/face-mask-detection/images")]
output_data_path = '.'

In [2]:
def parse_annotation_object(annotation_object):
    params = {}
    for param in list(annotation_object):
        if param.tag == 'name':
            params['name'] = param.text
        if param.tag == 'bndbox':
            for coord in list(param):
                params[coord.tag] = int(coord.text)
    return params

In [3]:
dataset = []
for anno in glob.glob(annotations_path + "/*.xml"):
    tree = ET.parse(anno)
    root = tree.getroot()
    constants = {'file': root.find('filename').text[0:-4]}
    objects = root.findall('object')
    for obj in objects:
        object_params = parse_annotation_object(obj)
        dataset.append({**constants, **object_params})

df = pd.DataFrame(dataset)

In [4]:
final_test_image = 'maksssksksss0'
images.remove(f'{final_test_image}.png')
df = df[df["file"] != final_test_image]

In [6]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=42)

In [7]:
for label in df['name'].unique():
    for d in ['train', 'test', 'val']:
        path = os.path.join(output_data_path, d, label)
        os.makedirs(path, exist_ok=True)

In [8]:
def crop_img(image_path, x_min, y_min, x_max, y_max):
    img = Image.open(image_path)
    cropped = img.crop((x_min - (x_max - x_min) * 0.1, y_min - (y_max - y_min) * 0.1, x_max + (x_max - x_min) * 0.1, y_max + (y_max - y_min) * 0.1))
    return cropped

def save_image(image, image_name, dataset_type, label):
    output_path = os.path.join(output_data_path, dataset_type, label, f'{image_name}.png')
    image.save(output_path)

for dataset, dataset_type in [(train_df, 'train'), (test_df, 'test'), (val_df, 'val')]:
    for _, row in dataset.iterrows():
        image_path = os.path.join(input_data_path, row['file'] + '.png')
        image = crop_img(image_path, row['xmin'], row['ymin'], row['xmax'], row['ymax'])
        save_image(image, row['file'] + '_' + str((row['xmin'], row['ymin'])), dataset_type, row['name'])


In [9]:
model = Sequential([
    Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(35, 35, 3)),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(units=500, activation='relu'),
    Dropout(0.2),
    Dense(units=3, activation='softmax')
])

In [10]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [11]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255, horizontal_flip=True, zoom_range=0.1, shear_range=0.2, width_shift_range=0.1,
    height_shift_range=0.1, rotation_range=4, vertical_flip=False
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [12]:
batch_size = 8
train_generator = datagen.flow_from_directory(directory='/kaggle/working/train', target_size=(35, 35),
                                              class_mode="categorical", batch_size=batch_size, shuffle=True)

val_generator = val_datagen.flow_from_directory(directory='/kaggle/working/val', target_size=(35, 35),
                                                class_mode="categorical", batch_size=batch_size, shuffle=True)

test_generator = val_datagen.flow_from_directory(directory='/kaggle/working/test', target_size=(35, 35),
                                                 class_mode="categorical", batch_size=batch_size, shuffle=False)

Found 1993 images belonging to 3 classes.
Found 855 images belonging to 3 classes.
Found 1221 images belonging to 3 classes.


In [13]:
model.fit_generator(generator=train_generator, epochs=45, validation_data=val_generator, callbacks=[])

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/45
250/250 [==============================] - 13s 17ms/step - loss: 0.3963 - accuracy: 0.8635 - val_loss: 0.2367 - val_accuracy: 0.9146
Epoch 2/45
250/250 [==============================] - 4s 16ms/step - loss: 0.2769 - accuracy: 0.9182 - val_loss: 0.2129 - val_accuracy: 0.9310
Epoch 3/45
250/250 [==============================] - 4s 15ms/step - loss: 0.2281 - accuracy: 0.9313 - val_loss: 0.2097 - val_accuracy: 0.9298
Epoch 4/45
250/250 [==============================] - 4s 16ms/step - loss: 0.2247 - accuracy: 0.9373 - val_loss: 0.1941 - val_accuracy: 0.9357
Epoch 5/45
250/250 [==============================] - 4s 15ms/step - loss: 0.2036 - accuracy: 0.9393 - val_loss: 0.1868 - val_accuracy: 0.9368
Epoch 6/45
250/250 [==============================] - 4s 15ms/step - loss: 0.2079 - accuracy: 0.9313 - val_loss: 0.1776 - val_accuracy: 0.9439
Epoch 7/45
250/250 [==============================] - 4s 15ms/step - loss: 0.2025 - accuracy: 0.9378 - val_loss: 0.1784 - val_accuracy: 0.936

In [14]:
model_loss, model_acc = model.evaluate(test_generator)
print(f'Test Loss: {model_loss}, Test Accuracy: {model_acc}')

153/153 [==============================] - 1s 7ms/step - loss: 0.2140 - accuracy: 0.9509
Test Loss: 0.21403174102306366, Test Accuracy: 0.9508599638938904
